# 1. IMPORTS

In [21]:
#data_propbankbr lives within datasets
import sys
sys.path.append('../datasets/')

import re
import numpy as np
import pandas as pd 
from data_propbankbr import propbankbr_parser, propbankbr_parser2, propbankbr_argument_stats, propbankbr_split

In [31]:
# This dataframe has all multiple arguments on columns ARG0 ~ARG6
synthdf = propbankbr_parser()
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5,ARG6
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,*,*,*,*,NaN,NaN,NaN
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,(A0*,*,*,*,NaN,NaN,NaN
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,*,*,*,*,NaN,NaN,NaN
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,*),*,*,*,NaN,NaN,NaN
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,(V*),*,*,*,NaN,NaN,NaN


In [34]:
dtreedf = propbankbr_parser2()
dtreedf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5,ARG6
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,-,-,-,-,None,None,None
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,A0,-,-,-,None,None,None
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,-,-,-,-,None,None,None
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,-,-,-,-,None,None,None
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,-,-,-,-,None,None,None


 ### 1. 1 Normalization
Convert each record into a machine learning example i.e one argument per row
* Filter        .: each ARG column
* Create        .: tmp row holding the number of the argument ( for further ordering )
* Concatenate   .: everybody afterwards by rows
* Sort          .: reorder by S and TMP


In [39]:
def unstack_df(df):
    '''
        Unstack outputs
        args:
            df :DataFrame either dependency or synthatic format
        returns:
            df :DataFrame
    '''
    columns_features= df.columns[:13]
    df_feature= df.loc[:,columns_features].copy()

    dataframes=[]
    num_records=0
    for i in range(7):
        col= 'ARG{:}'.format(i)
        df_target= df[col].dropna().to_frame()
        df_arg = df_feature.join(df_target, how='right')
        df_arg = df_arg.rename(index=str, columns={col: 'ARG'})
        df_arg['TMP']=i+1
    
        num_records+=df_arg.shape[0]
        print('{:}-\tnew records: {:}\ttotal records: {:}'.format(i, df_arg.shape[0],num_records))
        dataframes.append(df_arg)

    #Concatente, sort by sentence & argument, reindex
    df = pd.concat(dataframes,axis=0)
    df.sort_values(by=['S','TMP'],axis=0 ,kind='mergesort', inplace=True)    
    df= df.reset_index(drop=True)
    df.index.names=['INDEX']
    return df

In [40]:
synthdf = unstack_df(synthdf)

0-	new records: 69723	total records: 69723
1-	new records: 42159	total records: 111882
2-	new records: 19309	total records: 131191
3-	new records: 7114	total records: 138305
4-	new records: 2357	total records: 140662
5-	new records: 717	total records: 141379
6-	new records: 240	total records: 141619


In [42]:
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG,TMP
INDEX,,,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,*,1
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,(A0*,1
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,*,1
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,*),1
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,(V*),1


In [43]:
dtreedf = unstack_df(dtreedf)

0-	new records: 69723	total records: 69723
1-	new records: 42159	total records: 111882
2-	new records: 19309	total records: 131191
3-	new records: 7114	total records: 138305
4-	new records: 2357	total records: 140662
5-	new records: 717	total records: 141379
6-	new records: 240	total records: 141619


In [44]:
dtreedf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG,TMP
INDEX,,,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,-,1
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,A0,1
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,-,1
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,-,1
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,-,1


 ### 1.2 FIXING EXISTING FEATURES 
 #### 1.2.1 FIXING PREDICATE
  * __PRED__ must be non empty only when ARG is (V*) 
  * Drop column __TMP__

In [45]:
def fix_pred(df):
    index= (df['TMP'] == df['P_S'])
    df.loc[~index,'PRED']= '-'
    df = df.drop(labels='TMP', axis=1)
    return df

In [46]:
#Fix predicate 
synthdf = fix_pred(synthdf)
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG
INDEX,,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,*
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,(A0*
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,*
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,*)
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,(V*)


In [47]:
#Fix predicate 
dtreedf = fix_pred(dtreedf)
dtreedf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG
INDEX,,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,-
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,A0
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,-
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,-
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,-


 #### 1.2.2 FIXING P
  __P__ will be a range like id over the predicates.

In [48]:
def fix_p(df):
    P=[]
    P_S=[]
    S= list(set(df['S'].values))
    p=1

    for s in S:
        sindex=df['S'] == s
        ind= [int(x) for x in 
            df.loc[sindex,'ID'].values]
        p_s= [int(x) for x in 
            df.loc[sindex,'P_S'].values]
        l =max(ind)
        n_p= len(set(filter(lambda x : x > 0, p_s)))
        
        # redefining p_s
        for p_s in range(n_p):
            P+= l*[p]
            P_S+= l*[p_s]
            p+=1

    df['P']=P        
    df['P_S']=P_S        
    return df

            
    

In [49]:
synthdf = fix_p(synthdf)
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG
INDEX,,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,*
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,(A0*
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,*
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,*)
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,(V*)


In [50]:
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG
INDEX,,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,*
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,(A0*
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,*
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,*)
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,(V*)


In [51]:
synthdf.to_csv('../datasets/csvs/gs.csv', encoding='utf-8')

In [52]:
dtreedf = fix_p(dtreedf)
dtreedf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,IS_PRED,PRED,ARG
INDEX,,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),0,-,-
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,0,-,A0
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),0,-,-
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),0,-,-
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),1,revelar,-


In [53]:
dtreedf.to_csv('../datasets/csvs/gsdtree.csv', encoding='utf-8')